# **Recommender Systems**

Notebook created in PyTorch by [Paula G. Duran](https://www.linkedin.com/in/paulagd-1995/) for the UPC School (2020) and updated in (2022).


The problem we are trying to solve here is to predict a ranking for each user thus recommending a set of items from the huge number available on the original dataset. The dataset we will use is the ML-100K dataset, a classic dataset in the machine learning community when talking about recommendation.

## **Installation and imports**

In [ ]:
# Preparing imports
import torch
import pandas as pd
import numpy as np
import csv
import os
import scipy.sparse as sp

from typing import Tuple, Dict, Any, List
from tqdm import tqdm, trange
from IPython import embed
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter

In this session, I wanted to use the original Tensorboard instead of using the TensorboardColab version. Doing this, for example, we are able to add images or graphs and not just scalars. Besides, we are able to load different experiments on the same graphics thus allowing us to compare them in the same plot.

In [ ]:
%load_ext tensorboard 

In [ ]:
logs_base_dir = "runs"
os.makedirs(logs_base_dir, exist_ok=True)

In [ ]:
tb_fm = SummaryWriter(log_dir=f'{logs_base_dir}/{logs_base_dir}_FM/')
tb_rnd = SummaryWriter(log_dir=f'{logs_base_dir}/{logs_base_dir}_RANDOM/')

## **Setting up hyperparameters**

In [ ]:
# Let's define some hyper-parameters
hparams = {
    'batch_size':64,
    'num_epochs':20,
    'hidden_size': 32,
    'learning_rate':1e-4,
}

# we select to work on GPU if it is available in the machine, otherwise
# will run on CPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

## **Movielens - 100k dataset**

MovieLens [datasets](https://grouplens.org/datasets/movielens/) were collected by the GroupLens Research Project at the University of Minnesota.
 
&nbsp;


This data set consists of:

* 100,000 ratings (1-5) from 943 users on 1682 movies. 
* Each user has rated at least 20 movies. 
* Simple demographic info for the users (age, gender, occupation, zip)

 &nbsp;

The data was collected through the MovieLens web site (movielens.umn.edu) during the seven-month period from September 19th, 1997 through April 22nd, 1998. This data has been cleaned up - users who had less than 20 ratings or did not have complete demographic information were removed from this data set. 


> Note that the rating matrix is quite sparse (93.6% to be precise) as it only holds 100,000 ratings out of a possible 1,586,126 (943*1682).

&nbsp;



### **Downloading data and loading it with pandas ...**

In [ ]:
if not os.path.exists('ml-100k'):
    !wget "https://files.grouplens.org/datasets/movielens/ml-100k.zip"
    !unzip "ml-100k.zip"

The file `u.data` contains the entire data and it looks in the following way:


    user_id   movie_id   rating   timestamp

where `user_id` is an integer between 1 and 943, `movie_id` is an integer between 1 and 1682, `rating` is an integer between 1 and 5 and `timestamp`  is an epoch-based integer.

<div>
<center><img src="https://files.realpython.com/media/movielens-head.0542b4c067c7.jpg" width="300"/></center>
</div>

However, for this task we need to preprocess all data in order to convert all rating tags into binary labels to deal with this task from an `implicit feedback` point of view. So, all data from the dataset will have positive labels (`1`) denoting any interaction with a film as a case of being interesed in the film (even the user did not like it in the end). For negative labels we would have a (`0`) label but, as we can see below, we do not have any negative samples yet.






In [ ]:
def get_ml100k_dataset(data_path: str) -> pd.DataFrame:
    '''
    The dataset can be downloaded by doing
        - wget http://files.grouplens.org/datasets/movielens/ml-100k.zip
        - unzip ml-1m.zip -d ml-100k
    '''
    if not os.path.exists(os.path.join(data_path, 'interactions.csv')):
        '''
        Create a Pandas Data Frame for the "Users" csv file
        The column names are: user, gender, age, occupation, and zipcode
        '''
        
        df = pd.read_csv(os.path.join(data_path, "u.data"), delimiter='\t',
                         names=['user','item','rating','timestamp'])

        df['user'] = pd.Categorical(df['user']).codes
        df['item'] = pd.Categorical(df['item']).codes
        df['rating'] = 1

        df.to_csv(os.path.join(data_path, 'interactions.csv'), index=False)
    else:
        df = pd.read_csv(os.path.join(data_path, 'interactions.csv'))
    return df

df = get_ml100k_dataset(data_path='ml-100k')
df.head()

As we can observe below, the number of unique values for the label field is 1. This means we just have one type of data (positive data) and we will need to manually sample negative data.

 &nbsp;

As we just need to collect negative samples for training (and not for test), first we are going to split our dataset.


In [ ]:
df.nunique()

### **Splitting dataset (TLOO strategy)**

In this section we follow the time leave-one-out methodology to split the dataset by holding out the more recent interaction of each user.

In [ ]:
def split_train_test(data: np.ndarray,
                     n_users: int) -> Tuple[np.ndarray, np.ndarray]:
    # Split and remove timestamp
    train_x, test_x = [], []
    for u in trange(n_users, desc='spliting train/test and removing timestamp...'):
        user_data = data[data[:, 0] == u]
        sorted_data = user_data[user_data[:, -1].argsort()]
        if len(sorted_data) == 1:
            train_x.append(sorted_data[0][:-1])
        else:
            train_x.append(sorted_data[:-1][:, :-1])
            test_x.append(sorted_data[-1][:-1])
    return np.vstack(train_x), np.stack(test_x)

In [ ]:
data = df[['user', 'item', 'timestamp']].astype('int32').to_numpy()
data

> Note that we choose to preprocess the dataset by re-indexing the films in order to end up with a single identifier by entity (either user or item). Also, we will remove the timestamp from our data, as it is just useful for splitting data. 



In [ ]:
add_dims=0
for i in range(data.shape[1] - 1):  # do not affect to timestamp
    # MAKE IT START BY 0
    data[:, i] -= np.min(data[:, i])
    # RE-INDEX
    data[:, i] += add_dims
    add_dims = np.max(data[:, i]) + 1
dims = np.max(data, axis=0) + 1
data

In [ ]:
train_x, test_x = split_train_test(data, dims[0])

In [ ]:
print(f'Train shape: {train_x.shape}')
print(f'Test shape: {test_x.shape}')

In [ ]:
assert train_x.shape[0] + test_x.shape[0] == 100000

As we can observed, we have just held one interaction per user as a test set (the most recent one when sorting by timestamp), which will be used as ground-truth when evaluating the model bu outputing a ranking per user.

### **NEGATIVE SAMPLING**

As far as we know, a classifier needs both positive and negative data in order to learn. However, as our approach is to use implicit feedback we have converted any rating to the label `1`, and so No negative data is available in order to feed the model.

For that reason, we will need to perform negative sampling by sampling interactions that did not actually occured between a given user and a given item.

In [ ]:
train_x = train_x[:, :2]
dims = dims[:2]

In [ ]:
dims

In [ ]:
train_x

As we see, we are just taking the two first dimensions (user and item) to adress this problems. Hence, the `dims` variable contains the accumulated number of dimensions for each entity [#users, #(users+items)] because we have re-indexed them. By doing that, we get each user or item to have a unique identifier for the model. 


In order to be able to know which interactions did not occur we will build the rating matrix of this dataset. 

> Please note that the next function is extended to any extra dimension (such as context) that you might want to add.

In [ ]:
def build_adj_mx(n_feat:int, data:np.ndarray) -> sp.dok_matrix :
    train_mat = sp.dok_matrix((n_feat, n_feat), dtype=np.float32)
    for x in tqdm(data, desc=f"BUILDING ADJACENCY MATRIX..."):
        train_mat[x[0], x[1]] = 1.0
        train_mat[x[1], x[0]] = 1.0
        # IDEA: We treat features that are not user or item differently because we do not consider
        #  interactions between contexts
        if data.shape[1] > 2:
            for idx in range(len(x[2:])):
                train_mat[x[0], x[2 + idx]] = 1.0
                train_mat[x[1], x[2 + idx]] = 1.0
                train_mat[x[2 + idx], x[0]] = 1.0
                train_mat[x[2 + idx], x[1]] = 1.0
    return train_mat


Note that due to reindex, our adjacency matrix from training is even bigger. However, now it is even sparser. Storing this as a dense matrix would be a massive waste of both storage and computing power!
This is why use a scipy.lil_matrix sparse matrix for samples and a numpy array for labels.


Into the next function, we can observe how to perform negative sampling from the rating matrix.

In [ ]:
def ng_sample(data: np.ndarray, dims: list, num_ng:int=4) -> Tuple[np.ndarray, sp.dok_matrix]:
    rating_mat = build_adj_mx(dims[-1], data)
    interactions = []
    min_item, max_item = dims[0], dims[1]
    for num, x in tqdm(enumerate(data), desc='perform negative sampling...'):
        interactions.append(np.append(x, 1))
        for t in range(num_ng):
            j = np.random.randint(min_item, max_item) #if not pop else random.sample(items_to_sample, 1)[0]
            # IDEA: Loop to exclude true interactions (set to 1 in adj_train) user - item
            while (x[0], j) in rating_mat or j == int(x[1]):
                j = np.random.randint(min_item, max_item) #if not pop else random.sample(items_to_sample, 1)[0]
            interactions.append(np.concatenate([[x[0], j], x[2:], [0]]))
    return np.vstack(interactions), rating_mat


In [ ]:
train_x, rating_mat = ng_sample(train_x, dims)

In [ ]:
# Checking how sparse the rating matrix is
rating_mat

So, if we show the first 10 samples of our data after performing negative sampling we can observe that four negative samples have been introduced for each positive one. 

**Now we are ready to train any classifier!!!**

In [ ]:
train_x[:10]

### **Creating a dataset class**


We now create a Pytorch Dataset so that it is clear how we deal with the data. It could be a good practise to introduce the negative sampling inside this function if desired.

In [ ]:
class PointData(Dataset):
    def __init__(self,
                 data: np.ndarray,
                 dims: list) -> None:
        """
        Dataset formatter adapted point-wise algorithms
        Parameters
        """
        super(PointData, self).__init__()
        self.interactions = data
        self.dims = dims

    def __len__(self) -> int:
        return len(self.interactions)
        
    def __getitem__(self, 
                    index: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Return the pairs user-item and the target.
        """
        return self.interactions[index][:-1], self.interactions[index][-1]


In [ ]:
train_dataset = PointData(train_x, dims)

Now we already have the entire training set all setted up to be fed into a Pytorch Dataloader and train a model.

However, we still miss some part of our test set to be completed.

### **Preparing the test set for inference**

In [ ]:
test_x

If we plot the test_x set that we prepared we realize that we stored one sample for each user but, what does this sample actually mean?

Remaining the goal of RS, we should have in mind that we want to provide a personalized ranking for each user. To do so, we need to compare the score that a given user gets for interacting with each of the items from the database. Once we get the score of each item for a given user, we will be able to sort the scores out and provide an accurate ranking.

<div>
<center><img src="https://drive.google.com/uc?export=view&id=1DVIdCPnxNfpFLZHDrLWH_FJVrUvLMSti" width="700" /></center>
</div>


To do so, we need to add to each user test set also all the other items from the database even, of course, we will subtract the ones that the user interacted with in training. 

By doing so, we will expect the score of our ground-truth (GT) sample to be in the top@1 (hopefully) of the ranking. For example, in the image above we can see an example of the test set of the user one, where the higher score is actually the score of the GT sample and so our metrics would be amazing!!


In [ ]:
zero_positions = np.asarray(np.where(rating_mat.A==0)).T
zero_positions

Above we have asked for those coordinates from the rating matrix where we have 0-values. Thus, all the zeros of a given row will need to be candidates items over which we will need to perform the ranking.

In [ ]:
items2compute = []
for user in trange(dims[0]):
    aux = zero_positions[zero_positions[:, 0] == user][:, 1]
    items2compute.append(aux[aux >= dims[0]])

In [ ]:
items2compute[0]

In [ ]:
def build_test_set(itemsnoninteracted:list,
                   gt_test_interactions: np.ndarray) -> list:
    #max_users, max_items = dims # number users (943), number items (2625)
    test_set = []
    for pair, negatives in tqdm(zip(gt_test_interactions, itemsnoninteracted), desc="BUILDING TEST SET..."):
        # APPEND TEST SETS FOR SINGLE USER
        negatives = np.delete(negatives, np.where(negatives == pair[1]))
        single_user_test_set = np.vstack([pair, ] * (len(negatives)+1))
        single_user_test_set[:, 1][1:] = negatives
        test_set.append(single_user_test_set.copy())
    return test_set

In [ ]:
test_x = build_test_set(items2compute, test_x)

In [ ]:
test_x[0]

In the previous cell, we can observe the complete test set for the `user 1`, where the first pair refers to the GT sample (the film we know that `user 1` will watch next and that we expect to be in the first position of the ranking) and the other positions to any other item that the user did not consumed previously.


At this stage, we have both the training set and the test set setted up and ready to start training and evaluating the any model!! 

### **Building Factorization Machines model**

Now, before programming the training and the inference functions, we will explain a very common and used baseline as it is Factorization Machines. Please note that this model could be exchanged for any desired model by just changing the model class. Hence, all the pipeline regarding the data (defined previously) and also the one referring to the training and inference stages (which comes next) will still remind the same.


<div>
<center><img src="https://d2908q01vomqb2.cloudfront.net/f1f836cb4ea6efb2a0b1b99f41ad8b103eff4b59/2019/04/03/sagemaker-factorization-1.gif" width="400"/></center>
</div>

Looking at the formula above and the [FM paper](https://www.csie.ntu.edu.tw/~b97053/paper/Rendle2010FM.pdf) statements, we know that, as demonstrated in the paper, we can write down an implementation of the last term in the equation in the following way:

In [ ]:
# FM part of the equation
class FM_operation(torch.nn.Module):

    def __init__(self, 
                 reduce_sum: bool=True) -> None:
        super().__init__()
        self.reduce_sum = reduce_sum

    def forward(self,
                x: torch.Tensor) -> float:
        """
        :param x: Float tensor of size ``(batch_size, num_fields, embed_dim)``
        """
        square_of_sum = torch.sum(x, dim=1) ** 2
        sum_of_square = torch.sum(x ** 2, dim=1)
        ix = square_of_sum - sum_of_square
        if self.reduce_sum:
            ix = torch.sum(ix, dim=1, keepdim=True)
        return 0.5 * ix

Once we have clear how to implement the last term of the FM equation, we will now program the entire FM model by using the FM_operation class defined above.

In [ ]:
class FactorizationMachineModel(torch.nn.Module):
    """
    A pytorch implementation of Factorization Machine.

    Reference:
        S Rendle, Factorization Machines, 2010.
    """

    def __init__(self, 
                 field_dims: list,
                 embed_dim: float) -> None:
        super().__init__()
        self.linear = torch.nn.Linear(len(field_dims), 1)
        self.embedding = torch.nn.Embedding(field_dims[-1], embed_dim)
        self.fm = FM_operation(reduce_sum=True)

        torch.nn.init.xavier_uniform_(self.embedding.weight.data)

    def forward(self,
                interaction_pairs: torch.Tensor) -> torch.Tensor:
        """
        :param interaction_pairs: Long tensor of size ``(batch_size, num_fields)``
        """
        out = self.linear(interaction_pairs.float()) + self.fm(self.embedding(interaction_pairs))
        return out.squeeze(1)
        
    def predict(self, 
                interactions: np.ndarray,
                device: torch.device) -> torch.Tensor:
        # return the score, inputs are numpy arrays, outputs are tensors
        test_interactions = torch.from_numpy(interactions).to(dtype=torch.long, device=device) #, dtype=torch.long)
        output_scores = self.forward(test_interactions)
        return output_scores

### **Pipeline functions**

In this section we define both training and inference functions which are programmed to be called once per epoch.

#### **Training**

In [ ]:
from statistics import mean

def train_one_epoch(model: torch.nn.Module,
                    optimizer: torch.optim,
                    data_loader: torch.utils.data.DataLoader,
                    criterion: torch.nn.functional,
                    device: torch.device) -> float:
    model.train()
    total_loss = []

    for i, (interactions, targets) in enumerate(data_loader):
        interactions = interactions.to(device)
        targets = targets.to(device)

        predictions = model(interactions)
    
        loss = criterion(predictions, targets.float())
        model.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss.append(loss.item())

    return mean(total_loss)

#### **Inference**

In [ ]:
def test(model: torch.nn.Module,
         test_x: np.ndarray,
         device: torch.device,
         topk: int=10) -> Tuple[float, float]:
    # Test the HR and NDCG for the model @topK
    model.eval()

    HR, NDCG = [], []
    for user_test in test_x:
        gt_item = user_test[0][1]
        predictions = model.predict(user_test, device)
        _, indices = torch.topk(predictions, topk)
        recommend_list = user_test[indices.cpu().detach().numpy()][:, 1]

        HR.append(getHitRatio(recommend_list, gt_item))
        NDCG.append(getNDCG(recommend_list, gt_item))
    return mean(HR), mean(NDCG)

### **Define metrics**

Last but not least before start mounting the entire pipeline, we will define two metrics used in order to analyze the performance of the model: Hit Ratio and Normalized Cumulative Discounted Gain.

In [ ]:
import math

def getHitRatio(recommend_list: list,
                gt_item: int) -> bool:
    if gt_item in recommend_list:
        return 1
    else:
        return 0

def getNDCG(recommend_list: list,
            gt_item: int) -> float:
    idx = np.where(recommend_list == gt_item)[0]
    if len(idx) > 0:
        return math.log(2)/math.log(idx+2)
    else:
        return 0

### **PIPELINE**

#### **Defining the model, the loss and the optimizer**

In [ ]:
dims = train_dataset.dims
model = FactorizationMachineModel(dims, hparams['hidden_size']).to(device)

In [ ]:
criterion = torch.nn.BCEWithLogitsLoss(reduction='mean')
optimizer = torch.optim.Adam(params=model.parameters(), lr=hparams['learning_rate'])

#### **Random evaluation**

In this subsection we run an inference test (obviously without previously train the model) to check that everything works correctly and also observe the initial result, which will be equivalent to using a random model.

In [ ]:
import random
class RandomModel(torch.nn.Module):
    def __init__(self, 
                 dims: list) -> None:
        super(RandomModel, self).__init__()
        """
        Simple random based recommender system
        """
        self.all_items = list(range(dims[0], dims[1]))

    def forward(self) -> None:
        pass

    def predict(self,
                interactions: np.ndarray,
                device=None) -> torch.Tensor:
        return torch.FloatTensor(random.sample(self.all_items, len(interactions)))

rnd_model = RandomModel(dims)

#### **Final Pipeline**

In [ ]:
data_loader = DataLoader(train_dataset, batch_size=hparams['batch_size'], shuffle=True, num_workers=0)

#### **Start training the model**

In [ ]:
# DO EPOCHS NOW
topk = 10
for epoch_i in range(hparams['num_epochs']):
    #data_loader.dataset.negative_sampling()
    train_loss = train_one_epoch(model, optimizer, data_loader, criterion, device)
    hr, ndcg = test(model, test_x, device, topk=topk)

    print(f'epoch {epoch_i}:')
    print(f'training loss = {train_loss:.4f} | Eval: HR@{topk} = {hr:.4f}, NDCG@{topk} = {ndcg:.4f} ')
    print('\n')
 
    tb_fm.add_scalar('train/loss', train_loss, epoch_i)
    tb_fm.add_scalar('eval/HR@{topk}', hr, epoch_i)
    tb_fm.add_scalar('eval/NDCG@{topk}', ndcg, epoch_i)

    hr, ndcg = test(rnd_model, test_x, device, topk=topk)
    tb_rnd.add_scalar('eval/HR@{topk}', hr, epoch_i)
    tb_rnd.add_scalar('eval/NDCG@{topk}', ndcg, epoch_i)


## **Visualizing results**

Once we have trained both models (*fm with usual embbedding layers* vs *fm with embeddings from gcn*), we can observe both metrics and loss in the same graphic in order to compare:

In [ ]:
%tensorboard --logdir runs

# **QUESTIONNAIRE:**

- You can answer the test on the questionnaire section from MyTech.